# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
### Load dataset from pickle file

import pickle

training_file = 'train.p'
validation_file= 'valid.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

assert(len(X_train) == len(y_train))
assert(len(X_valid) == len(y_valid))
assert(len(X_test) == len(y_test))

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
import numpy as np

### Provide a basic summary of the dat set

# Number of training examples
n_train = len(X_train)

# Number of validation examples.
n_valid = len(X_valid)

# Number of testing examples.
n_test = len(X_test)

# What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(y_train))

print("Number of training examples =", n_train)
print("Number of validation examples =", n_valid)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization.

import random
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

# Visualize 10 images with labels
n_illustrations = 10
for i in range(0, n_illustrations):
    index = random.randint(0, len(X_train))
    image = X_train[index]
    plt.figure(figsize=(1,1))
    plt.imshow(image)
    print(y_train[index])
    plt.show()


----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Pre-process the Data Set (normalization, grayscale, etc.)

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. Preprocessing steps could include normalization, converting to grayscale, etc.
### Feel free to use as many code cells as needed.

import cv2
from sklearn.utils import shuffle
import random
from sklearn.model_selection import train_test_split


def convert_to_grayscale(image):
    """
    Convert RGB to Gray. 
    Reshape to (32,32,1) to fit the network input schema
    """
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).reshape((32,32,1))
    return gray_image

def normalize_grayscale(image):
    """
    Normalize gray scale from int [0:255] to float [0:1.0]
    """
    a = 0.0
    b = 1.0
    gray_min = 0
    gray_max = 255
    return a + (b - a)*(image - gray_min)/(gray_max - gray_min)

def add_gaussian_noises(image):
    """
    Add gaussian noises to the image
    """
    row, col, ch= image.shape
    mean = 0
    var = 0.01
    sigma = var**0.5
    gauss = np.random.normal(mean, sigma, (row,col,ch))
    gauss = gauss.reshape(row, col, ch)
    noisy = image + gauss
    return noisy

def rand_rotate(image):
    """
    Rotate the image by a random angel between -15 and +15.
    Reshape to (32,32,3) to fit the network input schema.
    """
    row = image.shape[0]
    col = image.shape[1]
    center=tuple(np.array([row,col])/2)
    angle = random.randint(-15,15)
    rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)
    new_image = cv2.warpAffine(image, rot_mat, (col,row)).reshape(32,32,3)
    return new_image

def pre_process_image(image):
    """
    Per evaluations:
        1. convert_to_grayscale does not help.
        2. normalize_grayscale helps. 
        3. add_guassian_noises helps when applying different noises per image per EPOCH
        4. rand_rotate helps when applying to a portion instead of all images in the dataset
    """
    # image = convert_to_grayscale(image)
    image = normalize_grayscale(image)
    # image = add_gaussian_noises(image)
    # image = rand_rotate(image)
    return image

# Apply the preprocessing (here it is only normalize_grayscale) to train, valid and test datasets. 
X_train = np.array([pre_process_image(image) for image in X_train])
X_valid = np.array([pre_process_image(image) for image in X_valid])
X_test = np.array([pre_process_image(image) for image in X_test])

# Augment the training dataset by rotating a portion of images to add to the training set. 
X_train_rotation = np.array([rand_rotate(image) for image in X_train])
X_rotation_keep, X_rotation_drop, y_rotation_keep, y_rotation_drop = train_test_split(
    X_train_rotation, y_train, test_size=0.66, random_state=59)
X_train = np.append(X_train, X_rotation_keep, axis=0)
y_train = np.append(y_train, y_rotation_keep, axis=0)
del X_train_rotation
del X_rotation_keep
del X_rotation_drop
del y_rotation_keep
del y_rotation_drop

### Model Architecture

In [ ]:
### Architecture definition

import tensorflow as tf
from tensorflow.contrib.layers import flatten

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    weights = {
        'conv_1': tf.Variable(tf.truncated_normal(shape = (5,5,3,6), mean = mu, stddev = sigma)),
        'conv_2': tf.Variable(tf.truncated_normal(shape = (5,5,6,16), mean = mu, stddev = sigma)),
        'full_1': tf.Variable(tf.truncated_normal(shape = (400,120), mean = mu, stddev = sigma)),
        'full_2': tf.Variable(tf.truncated_normal(shape = (120,84), mean = mu, stddev = sigma)),
        'full_3': tf.Variable(tf.truncated_normal(shape = (84,43), mean = mu, stddev = sigma))
    }
    
    biases = {
        'conv_1': tf.Variable(tf.zeros(6)),
        'conv_2': tf.Variable(tf.zeros(16)),
        'full_1': tf.Variable(tf.zeros(120)),
        'full_2': tf.Variable(tf.zeros(84)),
        'full_3': tf.Variable(tf.zeros(43))
    }
    
    ksizes = {
        'pool_1': [1, 2, 2, 1],
        'pool_2': [1, 2, 2, 1]
    }
    
    strides = {
        'conv_1': [1, 1, 1, 1],
        'conv_2': [1, 1, 1, 1],
        'pool_1': [1, 2, 2, 1],
        'pool_2': [1, 2, 2, 1]
    }
    
    paddings = {
        'conv_1': 'VALID',
        'conv_2': 'VALID',
        'pool_1': 'VALID',
        'pool_2': 'VALID'
    }

    # Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    conv_1 = tf.nn.conv2d(x, weights['conv_1'], strides['conv_1'], paddings['conv_1']) + biases['conv_1']
    conv_1 = tf.nn.relu(conv_1)
    # Pooling. Input = 28x28x6. Output = 14x14x6.
    pool_1 = tf.nn.max_pool(conv_1, ksizes['pool_1'], strides['pool_1'], paddings['pool_1'])
    
    # Layer 2: Convolutional. Output = 10x10x16.
    conv_2 = tf.nn.conv2d(pool_1, weights['conv_2'], strides['conv_2'], paddings['conv_2']) + biases['conv_2']
    conv_2 = tf.nn.relu(conv_2)
    # Pooling. Input = 10x10x16. Output = 5x5x16.
    pool_2 = tf.nn.max_pool(conv_2, ksizes['pool_2'], strides['pool_2'], paddings['pool_2'])

    # Flatten. Input = 5x5x16. Output = 400.
    flat_pool_2 = flatten(pool_2)
    
    # Layer 3: Fully Connected. Input = 400. Output = 120.
    full_1 = tf.matmul(flat_pool_2, weights['full_1']) + biases['full_1']
    full_1 = tf.nn.relu(full_1)

    # Layer 4: Fully Connected. Input = 120. Output = 84.
    full_2 = tf.matmul(full_1, weights['full_2']) + biases['full_2']
    full_2 = tf.nn.relu(full_2)
    
    # Layer 5: Fully Connected. Input = 84. Output = 43.
    logits = tf.matmul(full_2, weights['full_3']) + biases['full_3']
    
    return logits

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

#### Features and Labels

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)

#### Train Pipeline

In [ ]:
rate = 0.001

logits = LeNet(x)
predictions = tf.nn.softmax(logits)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

#### Evaluation

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

#### Training

In [ ]:
EPOCHS = 8
BATCH_SIZE = 128

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            batch_x = np.array([add_gaussian_noises(image) for image in batch_x])
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

#### Testing

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
import csv
import os

def get_signid_map(singsfile):
    """
    Map sign class names to sign class ids.
    """
    sign2id_map = {}
    with open(signsfile, 'rt') as signsf:
        reader = csv.reader(signsf)
        i = 0
        for line in reader:
            if i>0:
                sign2id_map[line[1]] = int(line[0])
            i += 1
    return sign2id_map

signsfile = 'signnames.csv'
signid_map = get_signid_map(signsfile)



In [ ]:
### Download images from the web, show the iamges, and get the test features and labels. 

from urllib.request import urlretrieve
from PIL import Image

def download(url, file):
    if not os.path.isfile(file):
        print('Start downloading ' + file + '...')
        urlretrieve(url, file)
        print('Downloading completed')
        
signurls = {
    'Go straight or right': 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Zeichen_214-20_-_Vorgeschriebene_Fahrtrichtung_geradeaus_und_rechts%2C_StVO_1992.svg/240px-Zeichen_214-20_-_Vorgeschriebene_Fahrtrichtung_geradeaus_und_rechts%2C_StVO_1992.svg.png',
    'Speed limit (60km/h)': 'https://wiki.openstreetmap.org/w/images/thumb/3/3d/Maxspeed.svg/200px-Maxspeed.svg.png',
    'Roundabout mandatory': 'https://upload.wikimedia.org/wikipedia/commons/thumb/7/72/Zeichen_215_-_Kreisverkehr%2C_StVO_2000.svg/240px-Zeichen_215_-_Kreisverkehr%2C_StVO_2000.svg.png',
    'End of all speed and passing limits': 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/03/Zeichen_282.svg/240px-Zeichen_282.svg.png',
    'Dangerous curve to the left': 'https://cdn.pixabay.com/photo/2013/07/13/13/15/dangerous-curve-to-the-left-160665_640.png'
}

net_features = []
net_labels = []

for sign, url in signurls.items():
    file_name = str(signid_map[sign]) + '.png'
    download(url, file_name)
    raw_image = Image.open(file_name).convert('RGB')
    raw_image.load()
    image = raw_image.resize((32, 32))
    feature = np.array(image, dtype=np.float32)
    label = int(os.path.split(file_name)[1].split('.')[0])
    net_features.append(feature)
    net_labels.append(label)
    print(sign)
    plt.imshow(image)
    plt.show()

### Predict the Sign Type for Each Image and Analyze Performance

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.


net_features = np.array([pre_process_image(image) for image in net_features])

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(net_features, net_labels)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

def get_top_classes(X_data, y_data):
    sess = tf.get_default_session()
    probs = sess.run(predictions, feed_dict={x: X_data, y: y_data})
    topkv = sess.run(tf.nn.top_k(tf.constant(probs), k=3))
    return topkv

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    topkv = get_top_classes(net_features, net_labels)
    print(topkv)





---

## Step 4: Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it maybe having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

### Question 9

Discuss how you used the visual output of your trained network's feature maps to show that it had learned to look for interesting characteristics in traffic sign images


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 